In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yasserh/breast-cancer-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/yasserh/breast-cancer-dataset/versions/1


In [3]:
import os
for file in os.listdir(path):
    print(file)

breast-cancer.csv


In [4]:
df = pd.read_csv(path+"/breast-cancer.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [5]:
df.shape

(569, 32)

In [6]:
df.drop(columns=['id'],inplace=True)
df = pd.DataFrame(df)  # Convert list to DataFrame (if applicable)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2, random_state=42)

In [8]:
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

In [9]:
encodor =LabelEncoder()
y_train = encodor.fit_transform(y_train)
y_test = encodor.transform(y_test)

In [11]:
y_train[0:10]

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 1])

In [12]:
X_train_tesnor = torch.tensor(X_train, dtype=torch.float32)
X_test_tnesor = torch.tensor(X_test, dtype=torch.float32)
y_train_tesnor = torch.tensor(y_train, dtype=torch.float32)
y_test_tesnor = torch.tensor(y_test, dtype=torch.float32)

In [14]:
from torch.utils.data import DataLoader, Dataset
class CostumDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [16]:
train_dataset = CostumDataset(X_train_tesnor, y_train_tesnor)
test_dataset = CostumDataset(X_test_tnesor, y_test_tesnor)

In [17]:
train_dataset[10]

(tensor([-0.4976,  0.6137, -0.4981, -0.5310, -0.5769, -0.1749, -0.3622, -0.2849,
          0.4335,  0.1782, -0.3684,  0.5531, -0.3167, -0.4052,  0.0403, -0.0380,
         -0.1804,  0.1648, -0.1217,  0.2308, -0.5004,  0.8194, -0.4692, -0.5331,
         -0.0491, -0.0416, -0.1491,  0.0968,  0.1062,  0.4904]),
 tensor(0.))

In [18]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [37]:
import torch.nn as nn
class MysimpleNN(nn.Module):
  def __init__(self , num_feature):
    super().__init__()
    self.linear = nn.Linear(num_feature,5)
    self.relu = nn.ReLU()
    self.linear2 = nn.Linear(5,1)
    self.sigmoid = nn.Sigmoid()
  def forward(self,features):
    out = self.linear(features)
    out = self.relu(out)
    out = self.linear2(out)
    out = self.sigmoid(out)
    return out

In [52]:
lr  = 0.001
epochs = 100

In [53]:
model = MysimpleNN(X_train_tesnor.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss()
loss_function = nn.BCELoss()

In [54]:
for epoch in range(epochs):
  for batch_features, batch_labels in train_loader:
    y_pred = model(batch_features)
    loss = loss_function(y_pred, batch_labels.view(-1, 1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.6389280557632446
Epoch: 2, Loss: 0.6320289969444275
Epoch: 3, Loss: 0.6259022355079651
Epoch: 4, Loss: 0.564919650554657
Epoch: 5, Loss: 0.4948568046092987
Epoch: 6, Loss: 0.3916046619415283
Epoch: 7, Loss: 0.5215939879417419
Epoch: 8, Loss: 0.2687731683254242
Epoch: 9, Loss: 0.23270049691200256
Epoch: 10, Loss: 0.41238221526145935
Epoch: 11, Loss: 0.17734012007713318
Epoch: 12, Loss: 0.19390378892421722
Epoch: 13, Loss: 0.3185284435749054
Epoch: 14, Loss: 0.15288949012756348
Epoch: 15, Loss: 0.20997540652751923
Epoch: 16, Loss: 0.19008390605449677
Epoch: 17, Loss: 0.08480048924684525
Epoch: 18, Loss: 0.09122419357299805
Epoch: 19, Loss: 0.03748440742492676
Epoch: 20, Loss: 0.06083548814058304
Epoch: 21, Loss: 0.03410021588206291
Epoch: 22, Loss: 0.19550064206123352
Epoch: 23, Loss: 0.02214205451309681
Epoch: 24, Loss: 0.08501984924077988
Epoch: 25, Loss: 0.2214914858341217
Epoch: 26, Loss: 0.17334342002868652
Epoch: 27, Loss: 0.45971640944480896
Epoch: 28, Loss: 0.12

In [41]:
model.